In [9]:
# Import libraries
import open3d as o3d
import numpy as np
from scipy.spatial import cKDTree
import dijkstra
import laspy
import os
import matplotlib.pyplot as plt

In [10]:
# Euclidean Distance Clustering Algorithm
def cluster_points(pcd, radius, min_cluster_size=1, max_cluster_size=10000):
    # kd tree
    kdtree = o3d.geometry.KDTreeFlann(pcd)  
    points_length = len(pcd.points)
    # Initializes a list of the same size as the point cloud, recording whether each point has been visited
    visited = [-1] * points_length
    clusters = []
    for index in range(points_length):
        # If the point has already been visited, skip it
        if visited[index] == 1:
            continue
        active_points = []
        active_index = 0
        # Mark the current point as visited and add it to the active point list
        active_points.append(index)
        visited[index] = 1
        while active_index < len(active_points):
            [k, indices, _] = kdtree.search_radius_vector_3d(pcd.points[active_points[active_index]],radius)
            if k == 1: 
                active_index += 1
                continue
            for i in range(k):
                if indices[i] == points_length or visited[indices[i]] == 1:
                    continue  
                active_points.append(indices[i])
                visited[indices[i]] = 1
            active_index += 1
        # If the cluster size is within the specified range, the cluster is logged
        if max_cluster_size > len(active_points) >= min_cluster_size:
            clusters.append(active_points)
    return clusters

In [12]:
def find_outside_points(points, radius, graph):
    tree = cKDTree(points)
    points_get = []
    # Iterate through the set of points to find the outermost edge points
    for i in range(len(points)):
        index = tree.query_ball_point(points[i], radius)
        point_need1 = points[np.where(points[index][:, 2] > points[i][2])]
        point_need2 = points[np.where(points[index][:, 2] < points[i][2])]
        if len(point_need1) == 0 or len(point_need2) == 0:
            points_get.append(points[i])
    array_affect = []
    z_points = sorted(points, key=(lambda z: z[2]))
    min_pts = z_points[0]
    # For each edge point, find the path to the lowest point
    for i in range(len(points_get)):
        path = get_path_points(points_get[i], min_pts, graph)
        array_affect.append(path)
    # Use a shortest path algorithm to remove incorrectly identified edge points
    iltered_array = remove_affect_points(np.asarray(array_affect,dtype=object))
    out = []
    for i in range(len(iltered_array)):
        out.append(iltered_array[i][0])
    return out

def clusters_to_pcd(pcd, dist, num):
    # Cluster the point cloud using the Euclidean distance clustering algorithm
    clusters = cluster_points(pcd, dist, min_cluster_size=num, max_cluster_size=100000)
    points = np.asarray(pcd.points)
    point_list = []
    # Iterate through each cluster, retrieve the points within, and add them to the list
    for i in clusters:
        point = points[i]
        point_list.append(point)
    concatenated_clusters = concatenate_points(point_list)
    pcd_trans = o3d.geometry.PointCloud()
    pcd_trans.points = o3d.utility.Vector3dVector(np.asarray(concatenated_clusters))
    return pcd_trans

def remove_support_structure_graph(pcd):
    pcd_trans = clusters_to_pcd(pcd, 0.5, 5)
    graph = make_graph(pcd_trans, 0.5)
    outside_points = find_outside_points(np.asarray(pcd_trans.points), 0.05, graph)
    z_points = sorted(np.asarray(pcd_trans.points), key=(lambda z: z[2]))
    min_pts = z_points[0]
    points = []
    # Get the shortest paths from the lowest point to all outermost points
    for i in range(len(outside_points)):
        path = get_path_points(min_pts, np.asarray(outside_points)[i], graph)
        points.append(path)
    concatenated_path = concatenate_points(points)
    concatenated_path_unique = np.unique(concatenated_path, axis=0)
    # Save the point cloud after removing the support structures
    pcd_remove = o3d.geometry.PointCloud()
    pcd_remove.points = o3d.utility.Vector3dVector(np.asarray(concatenated_path_unique))
    return pcd_remove, pcd_trans

In [13]:
def remove_support_structure_intensity(las_path,lidar_skeleton,lidar_pcd):
    las = laspy.read(las_path)
    intensity = las.intensity
    # Calculate the average intensity value of the entire point cloud
    intensity_mean_all = np.mean(intensity)
    points = np.asarray(lidar_skeleton.points)
    pcd_tree = o3d.geometry.KDTreeFlann(lidar_pcd)
    support_structure_points = []
    points_intensity_value = []
    # Compare the intensity value of each point with the average intensity of the entire point cloud
    for i in range(len(points)):
        [k, idx, d] = pcd_tree.search_radius_vector_3d(lidar_skeleton.points[i],0.05)
        points_intensity = intensity[idx]
        intensity_mean = np.mean(points_intensity)
        points_intensity_value.append(intensity_mean)
        if intensity_mean>intensity_mean_all * 1.1:#recommend:1.1-1.3
            support_structure_points.append(points[i])
    if len(support_structure_points) > 0:
        pcd_intensity = o3d.geometry.PointCloud()
        pcd_intensity.points = o3d.utility.Vector3dVector(np.asarray(support_structure_points))
        pcd_intensity.paint_uniform_color([1, 0.0, 0.0])
        return pcd_intensity,points_intensity_value
    elif len(support_structure_points) == 0:
        return None

In [14]:
def smooth_skeleton(start_point, end_point, current_point, iterations):
    # Smooth step size
    step_size = 0.1
    for i in range(iterations):
        direction_adjustment = (start_point - current_point) / 2 + (end_point - current_point) / 2
        # Adjust the position of the current point
        current_point = current_point + step_size * direction_adjustment
    return current_point

In [15]:
def repair_local(pcd_start,pcd1,max_path,radius,min_cluster):
    # Voxel downsampling
    pcd_down_sample = pcd1.voxel_down_sample(voxel_size = 0.03)
    pcd = clusters_to_pcd(pcd_start,radius,min_cluster)
    clusters = cluster_points(pcd,radius,min_cluster_size=min_cluster, max_cluster_size=100000)
     # Get each broken branch
    length = []
    for i in range(len(clusters)):
        length.append(len(clusters[i]))
    max_length = np.where(length==np.max(length))[0]
    clusters.remove(clusters[max_length[0]])
    pcd_max = o3d.geometry.PointCloud()
    pcd_max.points = o3d.utility.Vector3dVector(np.asarray(max_path))
    pcd_tree_max = o3d.geometry.KDTreeFlann(pcd_max)
    z_points = sorted(np.asarray(pcd_down_sample.points), key=(lambda z: z[2]))
    min_pts = sorted(z_points, key=(lambda z: z[2]))[0]
    nearest_points_list = []
    point_list = []
    # The nearest point to the most populous skeleton is the fracture point for each branch
    for i in range(len(clusters)):
        distance = []
        point = np.asarray(pcd.points)[clusters[i]]
        for j in range(len(point)):
            [k, idx, d] = pcd_tree_max.search_knn_vector_3d(point[j],1)
            distance.append(np.sqrt(d)[0])
        get_point = point[np.where(distance == np.amin(distance))][0]
        tree = cKDTree(point)
        point_index = tree.query_ball_point(get_point,0.02)
        point_need1 = point[np.where(point[point_index][:,2]>get_point[2])]
        point_need2 = point[np.where(point[point_index][:,2]<get_point[2])]
        if len(point_need1)==0 or len(point_need2)==0 :
            nearest_points_list.append(get_point)
            point_list.append(point)
    if len(nearest_points_list) > 0 :
        pcd_min_point = o3d.geometry.PointCloud()
        pcd_min_point.points = o3d.utility.Vector3dVector(np.asarray(nearest_points_list))
        # Select points from the downsampled point cloud for repair
        pcd_all = pcd_min_point + pcd_down_sample
        graph = make_graph(pcd_all,0.3)
        points_path_all = []
        for i in range(len(nearest_points_list)):
            path = get_path_points(np.asarray(nearest_points_list)[i],min_pts,graph)
            point_middle = np.mean(point_list[i],axis=0)
            middle_dist = np.linalg.norm(np.array(point_middle)-np.array(path[1]))
            middle_dist1 = np.linalg.norm(np.array(point_middle)-np.array(path[2]))
            if middle_dist1 > middle_dist:
                points_remain = remove_points_not_need(np.asarray(pcd.points),np.asarray(point_list[i]))
                pcd_remove_points = o3d.geometry.PointCloud()
                pcd_remove_points.points = o3d.utility.Vector3dVector(np.asarray(points_remain))
                pcd_tree_remove = o3d.geometry.KDTreeFlann(pcd_remove_points)
                points_path = []
                for l in range(len(path)):
                    [k, idx, d] = pcd_tree_remove.search_knn_vector_3d(path[l],1)
                    nearest_point =  np.asarray(pcd_remove_points.points)[idx][0]
                    [k1, idx1, d1] = pcd_tree_max.search_knn_vector_3d(path[l],1)
                    [k2, idx2, d2] = pcd_tree_max.search_knn_vector_3d(nearest_point,1)
                    nearest_dist = np.sqrt(d1) - np.sqrt(d2)
                    if (np.sqrt(d) > 0.05)  &(path[l][2]>0.45):
                        points_path.append(path[l])
                    elif (np.sqrt(d) < 0.05) & (path[l][2] > nearest_point[2]) & (nearest_dist > 0) :
                        points_path.append(path[l])
                    else:
                        break
                if len(points_path)>0:
                    points_path_all.append(points_path)
            else:
                continue
        if len(points_path_all)>0:
            # Smooth the selected point cloud
            points_smooth = []
            for i in range(len(points_path_all)):
                points_smooth1 = []
                for j in range(len(points_path_all[i])):
                    if len(points_path_all[i]) > 2:
                            if j ==1:
                                p = smooth_skeleton(np.array(points_path_all[i][j-1]),np.array(points_path_all[i][j+1]),np.array(points_path_all[i][j]),10)
                                points_smooth.append(p)
                                points_smooth1.append(p)
                            elif j == len(points_path_all[i])-1:
                                break
                            elif j > 1:
                                p = smooth_skeleton(np.array(points_path_all[i][j-1]),np.array(points_path_all[i][j+1]),p,10)
                                points_smooth.append(p)
            if len(points_smooth)>0:
                pcd_smooth = o3d.geometry.PointCloud()
                pcd_smooth.points = o3d.utility.Vector3dVector(np.asarray(points_smooth))
                #Add the smoothed point cloud to the skeleton point cloud
                pcd_whole = pcd_smooth + pcd
            else:
                pcd_whole = pcd
        else:
            pcd_whole = pcd
    else:
        pcd_whole = pcd
    return pcd_whole

In [16]:
def repair_tree_dijkstra(pcd_whole):
    points_all = []
    graph = make_graph(pcd_whole,0.3)
    z_points = sorted(np.asarray(pcd_whole.points), key=(lambda z: z[2]))
    min_pts = z_points[0]
    outside_points = find_outside_points(np.asarray(pcd_whole.points),0.03,graph)
    points_insert = []
     # Use linear interpolation to complete
    for i in range(len(np.asarray(outside_points))):
        path = get_path_points(np.asarray(min_pts),np.asarray(outside_points)[i],graph)
        step = 0.005
        for j in range(len(path)):
             if j != 0:
                dist = np.linalg.norm(np.array(path[j])-np.array(path[j-1]))
                # If the distance is greater than 0.015, then perform interpolation
                if dist >= 0.015:
                    number = np.floor(dist/step)
                    for k in range(int(number)):
                        point_need = np.asarray(path[j-1]) + k * step * (np.asarray(path[j])-np.asarray(path[j-1]))/dist
                        points_insert.append(point_need)
        points_all.append(points_insert)
        points_all.append(path)
    concatenated = concatenate_points(points_all)
    points_concatenate_remove = np.unique(concatenated,axis=0)
     # Obtain the completed point cloud
    pcd_finall_repair = o3d.geometry.PointCloud()
    pcd_finall_repair.points = o3d.utility.Vector3dVector(np.asarray(points_concatenate_remove))
    return pcd_finall_repair